In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

import json
import math
import os
import os.path as osp
import sys
import time
from datetime import datetime
from typing import Tuple, List, Iterable

import cv2
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
from PIL import Image
from matplotlib import rcParams
from matplotlib.axes import Axes
from pyquaternion import Quaternion
from tqdm import tqdm
#import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
# Use Agg backend for canvas
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

from nuscenes.lidarseg.lidarseg_utils import colormap_to_colors, plt_to_cv2, get_stats, \
    get_labels_in_coloring, create_lidarseg_legend, paint_points_label
from nuscenes.panoptic.panoptic_utils import paint_panop_points_label, stuff_cat_ids, get_frame_panoptic_instances,\
    get_panoptic_instances_stats
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud, Box
from nuscenes.utils.data_io import load_bin_file, panoptic_to_lidarseg
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.map_mask import MapMask
from nuscenes.utils.color_map import get_colormap

from sklearn.neighbors import KDTree

nusc = NuScenes(version='v1.0-mini', dataroot='/home/slava/Source/CV_Course/Course/nuscenes', verbose=True)

In [ ]:
nusc.list_scenes()

In [ ]:
my_scene = nusc.scene[4]
first_sample_token = my_scene['first_sample_token']

In [ ]:
# # Euclidean 2d clusterize

# class PCD_cluster:
#     def __init__(self, pcd):
#         self.pcd_2d = to_2d_pcd(pcd)
#         self.pcd_3d = to_3d_pcd(self.pcd_2d)
#         self.tree = KDTree(self.pcd_2d, leaf_size=int(self.pcd_2d.shape[0]*0.6))

#     def to_2d_pcd(self, pcd): # Work good
#         return np.delete(pcd, 2, 1)

#     def to_3d_pcd(self, pcd): # Work good
#         return np.hstack((pcd,np.zeros((pcd.shape[0],1))))
#         # Это нужно, чтобы каждой точке добавить метку кластера

#     def render_2d_pcd(self, pcd, is_centroids = False): # Work good
#         COSTMAP_RANGE = 50  # meters
#         COSTMAP_RESOLUTION = (800, 800)  # (w,h) pixels
    
#         costmap = np.zeros((COSTMAP_RESOLUTION[1], COSTMAP_RESOLUTION[0]), dtype=np.uint8)

#         distances = np.sqrt(pcd[:, 0] ** 2 + pcd[:, 1] ** 2)
#         close_points = pcd[distances < COSTMAP_RANGE]

#         costmap_shape = np.array(COSTMAP_RESOLUTION)
        
#         if is_centroids == True:
#             pixel_scale = ((costmap_shape//2) / COSTMAP_RANGE)*20
#         else:
#             pixel_scale = ((costmap_shape//2) / COSTMAP_RANGE)

#         costmap_coords = (close_points[:] * pixel_scale).astype(np.int32)#, :2
#         costmap_coords = costmap_coords + costmap_shape//2

#         costmap[costmap_coords[:, 0], costmap_coords[:, 1]] = 255

#         return costmap
    
#     def clusterize_point(self, radius, min_points_in_cluster, i, current_claster_number): 
#         if self.pcd_3d[i][2] == 0.0:
#             # Кластеризуем данную точку, так как она еще не кластеризована
#             self.pcd_3d[i][2] = current_claster_number
#             if self.tree.query_radius(self.pcd_2d[i:i+1], r=radius, count_only=True) > min_points_in_cluster: 
#                 indexes = self.tree.query_radius(self.pcd_2d[i:i+1], r=radius)
#                 for index in indexes:
#                     self.clusterize_point(radius, min_points_in_cluster, index, current_claster_number)
                    
#     def find_cluster_centroid(self, i): 
#         i = float(i)
#         indexes = (self.pcd_3d[:,2] == i)
#         cluster_pcd = self.pcd_2d[indexes]
        
#         signedArea = 0.0
#         cx = 0.0
#         cy = 0.0
#         j = 0
        
#         for k in range(len(cluster_pcd)-1):
#             j = j + 1
#             x0 = self.pcd_2d[k][0]
#             y0 = self.pcd_2d[k][1]
#             x1 = self.pcd_2d[k+1][0]
#             y1 = self.pcd_2d[k+1][1]
#             a = x0*y1 - x1*y0
#             signedArea = signedArea + a
#             cx = cx + (x0 + x1)*a;
#             cy = cy + (y0 + y1)*a;
            
#         x0 = self.pcd_2d[j][0]
#         y0 = self.pcd_2d[j][1]
#         x1 = self.pcd_2d[0][0]
#         y1 = self.pcd_2d[0][1]
#         a = x0*y1 - x1*y0;
#         signedArea = signedArea + a;
#         cx = cx + (x0 + x1)*a;
#         cy = cy + (y0 + y1)*a;
        
#         signedArea = signedArea * 0.5;
#         cx = cx / (6.0*signedArea);
#         cy = cy / (6.0*signedArea);
        
#         return [cx, cy]

#     def euclidean_clustering(self, radius, min_points_in_cluster): 
#         #pcd_3d = to_3d_pcd(pcd_2d)# здесь в 3 столбце будет храниться идентификатор кластера
#         # 0 = осначает отсутствие кластера, так что нумерация кластеров начнется с 1
#         current_claster_number = 1.0 
    
#         for i in range(len(self.pcd_2d)):
#             if self.pcd_3d[i][2] == 0.0:
#                 #print(1)
#                 if self.tree.query_radius(self.pcd_2d[i:i+1], r=radius, count_only=True) > min_points_in_cluster:
#                     self.clusterize_point(radius, min_points_in_cluster, i, current_claster_number)
#                     current_claster_number = current_claster_number + 1.0
                    
#         pcd_centroids = []
#         for i in range(1, int(current_claster_number) + 1):
#             cluster_centroid = find_cluster_centroid(i)
#             pcd_centroids.append(cluster_centroid)
            
#         return pcd_centroids
    
    
            

# my_pcds = get_pcds(my_scene)

# my_pcd0 = my_pcds[0]['pcd']

# pcd_cl = PCD_cluster(pcd=my_pcd0)
# cents = pcd_cl.euclidean_clustering(radius=1, min_points_in_cluster=10)
# image = pcl.render_2d_pcd(pcd = cents, is_centroids = True)

# plt.imshow(image)
# plt.title('image')
        
# plt.show()

# # # print(my_pcd0)
# #x = to_2d_pcd(my_pcd0)
# # y = to_3d_pcd(x)
# # print(y)
# # # print(x)
# #euclidean_clustering(x, radius = 1, min_points_in_cluster = 4)

# # image = render_2d_pcd(x)
# # plt.imshow(image)
# # plt.title('image')
        
# # plt.show()

In [ ]:
import numpy as np
from sklearn.neighbors import KDTree
rng = np.random.RandomState(0)
X = rng.random_sample((10,3 ))  # 10 points in 3 dimensions
print(X)
print("----")
tree = KDTree(X, leaf_size=2)  
print(X[0:1])
dist, ind = tree.query(X[0:1], k=3)                
print(ind)  # indices of 3 closest neighbors
#[0 3 1]
print(dist)  # distances to 3 closest neighbors
#[0. 0.19662693 0.29473397]
print(len(X))
for i in range(2, len(X)+1):
    print (i)

In [ ]:
# Utils

def get_pcds(scene):
    pcds = list()
    current_sample_token = scene['first_sample_token']
    
    while current_sample_token != '':
        current_sample = nusc.get('sample', current_sample_token)
        
        #pcd here
        pointsensor_token = current_sample["data"]['LIDAR_TOP']
        pointsensor_path = nusc.get_sample_data_path(pointsensor_token)
        pc = LidarPointCloud.from_file(pointsensor_path)
        scan = np.fromfile(pointsensor_path, dtype=np.float32)
        scanr = scan.reshape((-1, 5))[:, :3]
        
        #rotation, translation here
        pose_data = nusc.get('sample_data', current_sample['data']['LIDAR_TOP'])
        pose = nusc.get('ego_pose', pose_data['ego_pose_token'])
        rotation = pose['rotation']
        translation = pose['translation']
        
        pcd_dict = {'pcd': scanr, 'rot': rotation, 'tr': translation}
        pcds.append(pcd_dict)
        current_sample_token = current_sample.get('next', '')    
        
    return pcds

def get_transform_matrix(translation: np.ndarray = np.array([0, 0, 0]),
                     rotation: Quaternion = Quaternion([1, 0, 0, 0]),
                     inverse: bool = False) -> np.ndarray:
        tm = np.eye(4)
        if inverse:
            rot_inv = rotation.rotation_matrix.T
            trans = np.transpose(-np.array(translation))
            tm[:3, :3] = rot_inv
            tm[:3, 3] = rot_inv.dot(trans)
        else:
            tm[:3, :3] = rotation.rotation_matrix
            tm[:3, 3] = np.transpose(np.array(translation))

        return tm

def find_min_distance(array, point):
    distances = np.linalg.norm(array-point, axis=1)
    min_index = np.argmin(distances)
    #print(f"the point is {array[min_index]}, at a distance of {distances[min_index]}, has index {min_index}")
    return distances[min_index]

def pcd_render(pcd):
    COSTMAP_RANGE = 50  # meters
    COSTMAP_RESOLUTION = (800, 800)  # (w,h) pixels
    
    costmap = np.zeros((COSTMAP_RESOLUTION[1], COSTMAP_RESOLUTION[0]), dtype=np.uint8)

    distances = np.sqrt(pcd[:, 0] ** 2 + pcd[:, 1] ** 2)
    close_points = pcd[distances < COSTMAP_RANGE]

    costmap_shape = np.array(COSTMAP_RESOLUTION)
    pixel_scale = (costmap_shape//2) / COSTMAP_RANGE

    costmap_coords = (close_points[:, :2] * pixel_scale).astype(np.int32)
    costmap_coords = costmap_coords + costmap_shape//2

    costmap[costmap_coords[:, 0], costmap_coords[:, 1]] = 255

    return costmap

def apply_trans_matrix(pcd, trans_matrix):
    for i in range(pcd.shape[0]): 
        point = pcd[i]
        new_point = trans_matrix.dot(np.array([point[0], point[1], point[2], 1]))[:3]
        pcd[i] = new_point
        
    return pcd

def tr_sub(current_tr, prev_tr):
    res = list()
    
    res.append(current_tr[0] - prev_tr[0])
    res.append(current_tr[1] - prev_tr[1])
    res.append(current_tr[2] - prev_tr[2])
    
    return res

#dists = []

def rot_diff(q1: Quaternion = Quaternion([1, 0, 0, 0]), q2: Quaternion = Quaternion([1, 0, 0, 0])):
    inv = q1.inverse
    diff = q2*inv
    return diff
    

def pcd_filter(pcds):
    # create OpenCV video writer
    #out = cv2.VideoWriter("outpy.avi", cv2.VideoWriter_fourcc('M','J','P','G'), 2, (1600,800))
    #fig = plt.figure()
    frames = [] # for storing the generated images
    fig = plt.figure()
    
    for i in range(1, len(pcds)):
        current_pcd_dict = pcds[i]
        prev_pcd_dict = pcds[i-1]
        
        current_pcd = current_pcd_dict['pcd']
        current_rot = current_pcd_dict['rot']
        current_rot_q = Quaternion(current_rot)
        current_tr = current_pcd_dict['tr']
        
        prev_pcd = prev_pcd_dict['pcd']
        prev_rot = prev_pcd_dict['rot']
        prev_rot_q = Quaternion(prev_rot)
        prev_tr = prev_pcd_dict['tr']
        
        #quant(prev_rot)*inv(quant(current_rot))
        #delta_rot_q = current_rot_q - prev_rot_q
        delta_rot_q = rot_diff(current_rot_q, prev_rot_q)
        delta_tr = tr_sub(current_tr, prev_tr)
        
        trans_matrix = get_transform_matrix(translation = delta_tr, rotation = delta_rot_q)
        
        before = pcd_render(current_pcd)
        before = cv2.putText(before, 'BEFORE '+str(len(current_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        current_pcd = round_filter(current_pcd)
        current_pcd = rare_point_filter(current_pcd)
        
        prev_pcd = round_filter(prev_pcd)
        prev_pcd = rare_point_filter(prev_pcd)
        
        current = pcd_render(current_pcd)
        current = cv2.putText(current, 'CURRENT '+str(len(current_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        prev = pcd_render(prev_pcd)
        prev = cv2.putText(prev, 'PREV '+str(len(prev_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cp = np.concatenate((current, prev), axis=1)
        transed_prev_pcd = apply_trans_matrix(prev_pcd, trans_matrix)
        
        #print(str(len(transed_prev_pcd)) + "_" + str(len(current_pcd)))
        
        diff_pcd = distance_filter(current_pcd, transed_prev_pcd, 0.2, 3)
        #diff_pcd = rare_point_filter(diff_pcd)
        #print(len(diff_pcd))
                
        after = pcd_render(diff_pcd)
        after = cv2.putText(after, 'AFTER '+str(len(diff_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        vis = np.concatenate((before, after), axis=1)
        vis = np.concatenate((vis, cp), axis=0)
        vis = cv2.resize(vis, (1600,1600), interpolation = cv2.INTER_AREA)
        
        plot_hist = False
        
        if plot_hist == True:
            
            dists = []
            for i in range(len(current_pcd)):
                point = current_pcd[i]
                dist = find_min_distance(transed_prev_pcd, point)
                dists.append(dist)
    
            sdists = sorted(dists)
            #n, bin, patches = plt.hist(sdists, bins=500)
            #fig.canvas.draw()
            #img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
            #img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            #img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
            #cv2.imwrite("hist.png", img)
            #mat = cv2.resize(mat, (1600,400), interpolation = cv2.INTER_AREA)
            #print(mat.shape)
            #cv2.imshow(mat)
            #vis = np.concatenate((after, mat), axis=0)
            

        frames.append([plt.imshow(vis, animated=True)])#cmap=cm.Greys_r,        
        print(i)
        

    ani = animation.ArtistAnimation(fig, frames, interval=500, blit=False, repeat_delay=1000)
    ani.save('movie.mp4')
    plt.show()
    
def next_pcd_filter(pcds):
    # create OpenCV video writer
    #out = cv2.VideoWriter("outpy.avi", cv2.VideoWriter_fourcc('M','J','P','G'), 2, (1600,800))
    #fig = plt.figure()
    frames = [] # for storing the generated images
    fig = plt.figure()
    
    for i in range(0, len(pcds)-1):
        current_pcd_dict = pcds[i]
        next_pcd_dict = pcds[i+1]
        
        current_pcd = current_pcd_dict['pcd']
        current_rot = current_pcd_dict['rot']
        current_rot_q = Quaternion(current_rot)
        current_tr = current_pcd_dict['tr']
        
        next_pcd = next_pcd_dict['pcd']
        next_rot = next_pcd_dict['rot']
        next_rot_q = Quaternion(next_rot)
        next_tr = next_pcd_dict['tr']
        
        #quant(prev_rot)*inv(quant(current_rot))
        #delta_rot_q = current_rot_q - prev_rot_q
        delta_rot_q = rot_diff(current_rot_q, next_rot_q)
        delta_tr = tr_sub(current_tr, next_tr)
        
        trans_matrix = get_transform_matrix(translation = delta_tr, rotation = delta_rot_q)
        
        before = pcd_render(current_pcd)
        before = cv2.putText(before, 'BEFORE '+str(len(current_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        current_pcd = round_filter(current_pcd)
        current_pcd = rare_point_filter(current_pcd)
        
        next_pcd = round_filter(next_pcd)
        next_pcd = rare_point_filter(next_pcd)
        
        current = pcd_render(current_pcd)
        current = cv2.putText(current, 'CURRENT '+str(len(current_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        nextn = pcd_render(next_pcd)
        nextn = cv2.putText(nextn, 'NEXT '+str(len(next_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cp = np.concatenate((current, nextn), axis=1)
        #transed_next_pcd = apply_trans_matrix(next_pcd, trans_matrix)
        transed_current_pcd = apply_trans_matrix(current_pcd, trans_matrix)
        
        #print(str(len(transed_prev_pcd)) + "_" + str(len(current_pcd)))
        
        diff_pcd = distance_filter(next_pcd, transed_current_pcd, 0.2, 3)
        #diff_pcd = rare_point_filter(diff_pcd)
        #print(len(diff_pcd))
                
        after = pcd_render(diff_pcd)
        after = cv2.putText(after, 'AFTER '+str(len(diff_pcd)), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        vis = np.concatenate((before, after), axis=1)
        vis = np.concatenate((vis, cp), axis=0)
        vis = cv2.resize(vis, (1600,1600), interpolation = cv2.INTER_AREA)
        
#         plot_hist = False
        
#         if plot_hist == True:
            
#             dists = []
#             for i in range(len(current_pcd)):
#                 point = current_pcd[i]
#                 dist = find_min_distance(transed_next_pcd, point)
#                 dists.append(dist)
    
#             sdists = sorted(dists)
            #n, bin, patches = plt.hist(sdists, bins=500)
            #fig.canvas.draw()
            #img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
            #img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            #img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
            #cv2.imwrite("hist.png", img)
            #mat = cv2.resize(mat, (1600,400), interpolation = cv2.INTER_AREA)
            #print(mat.shape)
            #cv2.imshow(mat)
            #vis = np.concatenate((after, mat), axis=0)
            

        frames.append([plt.imshow(vis, animated=True)])#cmap=cm.Greys_r,        
        print(i)
        

    ani = animation.ArtistAnimation(fig, frames, interval=500, blit=False, repeat_delay=1000)
    ani.save('movie.mp4')
    plt.show()
        
def round_filter(pcd):
    indexes = (pcd[:,2] > 0.01)# & (pcd[:,2] < 3)
    pcd = pcd[indexes]
    
    return pcd

def distance_filter(next_pcd, current_pcd, tresh_min, tresh_max):
    dists = []

    for i in range(len(current_pcd)):
        point = current_pcd[i]
        dist = find_min_distance(next_pcd, point)
        dists.append(dist)
    
    #sdists = sorted(dists)
    indexes = [i for i in range(len(dists)) if tresh_min < dists[i] and dists[i] < tresh_max]
    #indexes = (dists[:1] > tresh)
    current_pcd = current_pcd[indexes]
    
    return current_pcd
        
def rare_point_filter(pcd):
    tree = KDTree(pcd, leaf_size=int(pcd.shape[0]*0.6))
    tree_q = tree.query_radius(pcd, r=1, count_only=True)
    pcd = (pcd[tree_q > 5])
    return pcd


In [ ]:
my_pcds = get_pcds(my_scene)
#pcd_filter(my_pcds)
next_pcd_filter(my_pcds)

In [ ]:
plt.rcParams["figure.figsize"] = (16,16)

In [ ]:
my_pcds = get_pcds(my_scene)

my_pcd0 = my_pcds[0]['pcd']

my_rot0 = Quaternion(my_pcds[0]['rot'])
my_tr0 = my_pcds[0]['tr']

my_rot1 = Quaternion(my_pcds[1]['rot'])
my_tr1 = my_pcds[1]['tr']

#my_rot = my_rot1 - my_rot0
my_rot = rot_diff(my_rot1, my_rot0)
my_tr = tr_sub(my_tr1, my_tr0)

tr_mat = get_transform_matrix(translation = my_tr, rotation = my_rot)#, rotation = my_rot
my_pcd0 = apply_trans_matrix(my_pcd0, tr_mat)
#my_pcd0 = round_filter(my_pcd0)
my_pcd1 = my_pcds[1]['pcd']
#my_pcd0
print(my_pcd0)
print(my_pcd1)

In [ ]:
dists = []

for i in range(len(my_pcd1)):
    point = my_pcd1[i]
    dist = find_min_distance(my_pcd0, point)
    dists.append(dist)
    
sdists = sorted(dists)

In [ ]:
n, bin, patches = plt.hist(sdists, bins=500)
plt.show()
# print(n)
# print(bin)
# print(patches)

In [ ]:
c_pcd = distance_filter(my_pcd1, my_pcd0, 2)
imagex = pcd_render(c_pcd)
plt.imshow(imagex)
plt.title('imagex')
        
plt.show()

In [ ]:
image0 = pcd_render(my_pcd0)
plt.imshow(image0)
plt.title('image0')
        
plt.show()

In [ ]:
image1 = pcd_render(my_pcd1)
plt.imshow(image1)
plt.title('image1')
        
plt.show()